In [16]:
import pandas as pd
import sys
import praw
import json
import requests
import time
from datetime import datetime, timedelta

In [4]:
reddit = praw.Reddit(
    client_id="pt5Jv1sjiMJcNQ",
    client_secret="szxNI44AB0JiI0BF5g6au4SFdsTV2A",
    user_agent="my user agent"
)

Version 7.1.2 of praw is outdated. Version 7.1.4 was released Sunday February 07, 2021.


In [5]:
subreddit_female = reddit.subreddit("femalefashionadvice")
subreddit_male = reddit.subreddit("malefashionadvice")

In [60]:
def submissions_pushshift_praw(subreddit, start=None, end=None, limit=100, extra_query=""):
    """
    A simple function that returns a list of PRAW submission objects during a particular period from a defined sub.
    This function serves as a replacement for the now deprecated PRAW `submissions()` method.
    
    :param subreddit: A subreddit name to fetch submissions from.
    :param start: A Unix time integer. Posts fetched will be AFTER this time. (default: None)
    :param end: A Unix time integer. Posts fetched will be BEFORE this time. (default: None)
    :param limit: There needs to be a defined limit of results (default: 100), or Pushshift will return only 25.
    :param extra_query: A query string is optional. If an extra_query string is not supplied, 
                        the function will just grab everything from the defined time period. (default: empty string)
    
    Submissions are yielded newest first.
    
    For more information on PRAW, see: https://github.com/praw-dev/praw 
    For more information on Pushshift, see: https://github.com/pushshift/api
    """
    matching_praw_submissions = []
    
    # Default time values if none are defined (credit to u/bboe's PRAW `submissions()` for this section)
    utc_offset = 28800
    now = int(time.time())
    start = max(int(start) + utc_offset if start else 0, 0)
    end = min(int(end) if end else now, now) + utc_offset
    
    # Format our search link properly.
    search_link = ('https://api.pushshift.io/reddit/submission/search/'
                   '?subreddit={}&after={}&before={}&sort_type=score&sort=asc&limit={}&q={}')
    search_link = search_link.format(subreddit, start, end, limit, extra_query)
    
    # Get the data from Pushshift as JSON.
    retrieved_data = requests.get(search_link)
    returned_submissions = retrieved_data.json()['data']
    
    # Iterate over the returned submissions to convert them to PRAW submission objects.
    for submission in returned_submissions:
        
        # Take the ID, fetch the PRAW submission object, and append to our list
        praw_submission = reddit.submission(id=submission['id'])
        
        if ((praw_submission.link_flair_text != '[Daily]') & (praw_submission.link_flair_text != '[Weekly]')):
            matching_praw_submissions.append(praw_submission)
     
    # Return all PRAW submissions that were obtained.
    return matching_praw_submissions

In [84]:
submissions = []
for submission in submissions_pushshift_praw('femalefashionadvice', 1581851510, 1613473910, 10):
    if (submission.selftext == '[removed]') | (submission.selftext == '[deleted]'):
        continue
    json_output = {}
    json_output['title'] = submission.title
    json_output['body'] = submission.selftext
    json_output['score'] = submission.score
    json_output['id'] = submission.id
    json_output['author'] = submission.author.name
    json_output['ups'] = submission.ups
    json_output['downs'] = submission.downs
    json_output['date'] = submission.created
    json_output['comments'] = []
    
    
    submission.comments.replace_more(limit=None)
    
    comments = submission.comments.list()
    
    for comment in submission.comments:
        comment_dict = {}
        comment_dict['author'] = comment.author.name
        comment_dict['id'] = comment.id
        comment_dict['body'] = comment.body
        comment_dict['score'] = comment.score
        comment_dict['ups'] = comment.ups
        comment_dict['downs'] = comment.downs
        comment_dict['date'] = comment.created
        comment_dict['depth'] = comment.depth
        comments = get_all_replies(comment)
        comment_dict['replies'] = comments
        json_output['comments'].append(comment_dict)
    submissions.append(json_output)
print(json.dumps(submissions, indent=3))

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readt

[
   {
      "title": "Outfit Challenge - Star Wars Inspired",
      "body": "",
      "score": 10,
      "id": "ghtd3e",
      "author": "chameshi_nampa",
      "ups": 10,
      "downs": 0,
      "date": 1589250643.0,
      "comments": []
   },
   {
      "title": "Can only rich people be fashion influencers on instagram?",
      "body": "idk if this is right place to ask that here but do you think the same too? I believe i have a decent to good fashion taste and if i would have access to better quality clothes (or makeup) i would have definitely been an influencer. I am a college student from a upper middle class background I can afford clothes but i can't afford a lot of them. I noticed influencers have new outfits everyday. Location, theme, editing, photography is something that i can manage but the lack of quality and quantity in clothing puts me off.",
      "score": 0,
      "id": "hhla9l",
      "author": "tanmar11",
      "ups": 0,
      "downs": 0,
      "date": 1593403291.0,

In [83]:
def get_all_replies(comment):
    replies = []
    if comment.replies.__len__() != 0:
        for reply in comment.replies:
            reply_dict = {}
            reply_dict['author'] = reply.author.name
            reply_dict['id'] = reply.id
            reply_dict['body'] = reply.body
            reply_dict['score'] = reply.score
            reply_dict['ups'] = reply.ups
            reply_dict['downs'] = reply.downs
            reply_dict['date'] = reply.created
            reply_dict['depth'] = reply.depth
            reply_dict['replies'] = []
            replies.append(reply_dict)
            
            all_replies = get_all_replies(reply)
            if len(all_replies) > 0:
                replies[-1]['replies'] = all_replies
    return replies